In [203]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import re

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False)

INFO: Pandarallel will run on 72 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
used_car_data = pd.read_csv('used_cars_data.csv', low_memory=False)

In [3]:
used_car_data.columns

Index(['vin', 'back_legroom', 'bed', 'bed_height', 'bed_length', 'body_type',
       'cabin', 'city', 'city_fuel_economy', 'combine_fuel_economy',
       'daysonmarket', 'dealer_zip', 'description', 'engine_cylinders',
       'engine_displacement', 'engine_type', 'exterior_color', 'fleet',
       'frame_damaged', 'franchise_dealer', 'franchise_make', 'front_legroom',
       'fuel_tank_volume', 'fuel_type', 'has_accidents', 'height',
       'highway_fuel_economy', 'horsepower', 'interior_color', 'isCab',
       'is_certified', 'is_cpo', 'is_new', 'is_oemcpo', 'latitude', 'length',
       'listed_date', 'listing_color', 'listing_id', 'longitude',
       'main_picture_url', 'major_options', 'make_name', 'maximum_seating',
       'mileage', 'model_name', 'owner_count', 'power', 'price', 'salvage',
       'savings_amount', 'seller_rating', 'sp_id', 'sp_name', 'theft_title',
       'torque', 'transmission', 'transmission_display', 'trimId', 'trim_name',
       'vehicle_damage_category', 'whe

In [4]:
used_car_data.loc[used_car_data.back_legroom == '--', 'back_legroom'] = float('nan')

In [5]:
used_car_data.back_legroom.dropna().str.endswith(' in').all()

True

In [6]:
used_car_data.back_legroom = used_car_data.back_legroom.str.replace(' in', '').astype('float')

In [7]:
used_car_data.loc[used_car_data.bed_height == '--', 'bed_height'] = float('nan')

In [8]:
used_car_data.bed_height.dropna()

Series([], Name: bed_height, dtype: object)

In [9]:
used_car_data.loc[used_car_data.bed_length == '--', 'bed_length'] = float('nan')

In [10]:
used_car_data.bed_length.dropna().str.endswith(' in').all()

True

In [11]:
used_car_data.bed_length = used_car_data.bed_length.str.replace(' in', '').astype('float')

In [12]:
used_car_data.body_type.unique()

array(['SUV / Crossover', 'Sedan', 'Coupe', 'Hatchback', 'Pickup Truck',
       'Wagon', 'Minivan', 'Van', 'Convertible', nan], dtype=object)

In [13]:
used_car_data.cabin.unique()

array([nan, 'Crew Cab', 'Extended Cab', 'Regular Cab', 'Large Crew Cab'],
      dtype=object)

In [30]:
city_vc = used_car_data.city.value_counts()
top_cities = city_vc[city_vc>5000].index

In [31]:
city_data = []
for city in top_cities:
    entries = used_car_data[used_car_data.city==city]
    city_data += [[entries.latitude.median(), entries.longitude.median()]]
city_data = pd.DataFrame(city_data, columns=['latitude', 'longitude'], index=top_cities)

In [32]:
((city_data.latitude-32)**2+(city_data.longitude+96)**2).idxmin()

'Dallas'

In [33]:
top_city_set = set(top_cities)

In [34]:
mapped_cities = used_car_data.parallel_apply(lambda x: x.city if x.city in top_city_set else ((city_data.latitude-x.latitude)**2+(city_data.longitude-x.longitude)**2).idxmin(), axis=1)

In [35]:
mapped_cities

0            Miami
1            Miami
2            Miami
3            Miami
4            Miami
            ...   
3000035    Fremont
3000036    Fremont
3000037    Fremont
3000038    Fremont
3000039    Fremont
Length: 3000040, dtype: object

In [36]:
mapped_cities.value_counts()

Manchester     251277
Alexandria     138723
Chicago         95054
Cleveland       90823
Minneapolis     76758
                ...  
Scottsdale      10499
Chandler         9764
Mesa             8525
Englewood        6810
Gilbert          5783
Length: 92, dtype: int64

In [38]:
used_car_data.city[mapped_cities=='Los Angeles'].value_counts()

Los Angeles         5790
Cerritos            4284
Costa Mesa          3548
Huntington Beach    3476
Torrance            3275
Buena Park          2492
Westminster         2207
Signal Hill         1670
Bellflower          1279
Carson              1268
Long Beach          1192
Culver City         1068
Hawthorne            955
Inglewood            900
Gardena              864
Downey               737
Montebello           686
Garden Grove         627
Santa Ana            590
Norwalk              582
Fullerton            564
Beverly Hills        533
South Gate           518
Santa Fe Springs     377
Stanton              327
La Habra             326
Monterey Park        315
Whittier             308
Midway City          300
Commerce             266
Anaheim              265
Newport Beach        189
Manhattan Beach      173
Lynwood              149
Lawndale             141
Hermosa Beach        106
Cypress               76
Bell                  66
Redondo Beach         56
Marina Del Rey        50


In [39]:
used_car_data.city = mapped_cities

In [40]:
used_car_data.city_fuel_economy

0           NaN
1           NaN
2          17.0
3           NaN
4           NaN
           ... 
3000035    26.0
3000036    18.0
3000037     NaN
3000038    30.0
3000039    26.0
Name: city_fuel_economy, Length: 3000040, dtype: float64

In [42]:
used_car_data.combine_fuel_economy.dropna()

Series([], Name: combine_fuel_economy, dtype: float64)

In [43]:
used_car_data.daysonmarket

0           522
1           207
2          1233
3           196
4           137
           ... 
3000035      16
3000036     171
3000037      91
3000038      11
3000039      17
Name: daysonmarket, Length: 3000040, dtype: int64

In [46]:
used_car_data.engine_cylinders = used_car_data.engine_cylinders.str[:2]

In [50]:
used_car_data.engine_cylinders

0           I4
1           I4
2           H4
3           V6
4           I4
          ... 
3000035     I4
3000036     V6
3000037    NaN
3000038     I4
3000039     I4
Name: engine_cylinders, Length: 3000040, dtype: object

In [47]:
used_car_data.engine_displacement

0          1300.0
1          2000.0
2          2500.0
3          3000.0
4          2000.0
            ...  
3000035    1500.0
3000036    3600.0
3000037    2000.0
3000038    2000.0
3000039    2500.0
Name: engine_displacement, Length: 3000040, dtype: float64

In [52]:
used_car_data.fleet

0            NaN
1            NaN
2          False
3            NaN
4            NaN
           ...  
3000035     True
3000036      NaN
3000037    False
3000038    False
3000039    False
Name: fleet, Length: 3000040, dtype: object

In [54]:
used_car_data.frame_damaged = used_car_data.frame_damaged.fillna(False)

In [59]:
used_car_data.franchise_dealer

0           True
1           True
2           True
3           True
4           True
           ...  
3000035    False
3000036     True
3000037     True
3000038    False
3000039     True
Name: franchise_dealer, Length: 3000040, dtype: bool

In [61]:
used_car_data.franchise_make.value_counts()

Ford             395521
Chevrolet        338817
Toyota           186718
Honda            184515
Jeep             177100
Nissan           152590
Hyundai          107099
Kia               92690
RAM               77079
Buick             74419
Volkswagen        62430
Subaru            60899
GMC               60677
Dodge             49212
Mercedes-Benz     45867
Mazda             45743
BMW               45540
Cadillac          35081
Lexus             28602
Audi              26896
Acura             26282
Lincoln           25535
Mitsubishi        20079
Chrysler          19563
Volvo             19353
INFINITI          18665
Land Rover        12662
Porsche           10897
MINI               5448
Jaguar             4965
FIAT               4207
Scion              3976
Maserati           2598
Alfa Romeo         1486
Genesis            1123
Bentley             541
Rolls-Royce         506
Aston Martin        482
Ferrari             458
Lotus               436
McLaren             309
Lamborghini     

In [65]:
used_car_data.front_legroom = used_car_data.front_legroom.replace('--', float('nan'))

In [66]:
used_car_data.front_legroom.dropna().str.endswith(' in').all()

True

In [67]:
used_car_data.front_legroom = used_car_data.front_legroom.str.replace(' in', '').astype('float')

In [70]:
used_car_data.fuel_tank_volume = used_car_data.fuel_tank_volume.replace('--', float('nan'))

In [72]:
used_car_data.fuel_tank_volume.dropna().str.endswith(' gal').all()

True

In [73]:
used_car_data.fuel_tank_volume = used_car_data.fuel_tank_volume.str.replace(' gal', '').astype('float')

In [74]:
used_car_data.fuel_type

0          Gasoline
1          Gasoline
2          Gasoline
3          Gasoline
4          Gasoline
             ...   
3000035    Gasoline
3000036    Gasoline
3000037         NaN
3000038      Diesel
3000039    Gasoline
Name: fuel_type, Length: 3000040, dtype: object

In [76]:
used_car_data.has_accidents = used_car_data.has_accidents.fillna(False)

In [80]:
used_car_data.height = used_car_data.height.replace('--', float('nan'))
used_car_data.height.str.endswith(' in').all()

True

In [81]:
used_car_data.height = used_car_data.height.str.replace(' in', '').astype('float')

In [95]:
used_car_data.fleet = used_car_data.fleet.fillna(False)

In [101]:
used_car_data.highway_fuel_economy

0           NaN
1           NaN
2          23.0
3           NaN
4           NaN
           ... 
3000035    32.0
3000036    27.0
3000037     NaN
3000038    40.0
3000039    33.0
Name: highway_fuel_economy, Length: 3000040, dtype: float64

In [102]:
used_car_data.horsepower

0          177.0
1          246.0
2          305.0
3          340.0
4          246.0
           ...  
3000035    170.0
3000036    310.0
3000037    240.0
3000038    180.0
3000039    170.0
Name: horsepower, Length: 3000040, dtype: float64

In [104]:
used_car_data.isCab = used_car_data.isCab.fillna(False)

In [112]:
used_car_data.is_cpo = used_car_data.is_cpo.fillna(False)

In [114]:
used_car_data.is_new.hasnans

False

In [117]:
((used_car_data.is_oemcpo.fillna(False) | used_car_data.is_cpo) == used_car_data.is_cpo).all()

True

In [118]:
used_car_data.length = used_car_data.length.replace('--', float('nan'))
used_car_data.length.str.endswith(' in').all()

True

In [119]:
used_car_data.length = used_car_data.length.str.replace(' in', '').astype('float')

In [142]:
month_listed = used_car_data.listed_date.astype('datetime64').dt.month
year_listed = used_car_data.listed_date.astype('datetime64').dt.year

In [145]:
month_listed.value_counts()

8     1106388
9      565170
7      536309
6      225923
3      115727
2       93972
5       88574
1       69368
4       63353
12      57037
11      44123
10      34096
Name: listed_date, dtype: int64

In [153]:
used_car_data.maximum_seating = used_car_data.maximum_seating.replace('--', float('nan'))
used_car_data.maximum_seating.str.endswith(' seats').all()

True

In [157]:
used_car_data.maximum_seating = used_car_data.maximum_seating.str.replace(' seats', '').astype('float')

In [158]:
used_car_data.mileage

0              7.0
1              8.0
2              NaN
3             11.0
4              7.0
            ...   
3000035    41897.0
3000036        5.0
3000037    57992.0
3000038    27857.0
3000039    22600.0
Name: mileage, Length: 3000040, dtype: float64

In [160]:
used_car_data.owner_count = used_car_data.owner_count.fillna(1.)

In [162]:
used_car_data.price

0          23141.0
1          46500.0
2          46995.0
3          67430.0
4          48880.0
            ...   
3000035    17998.0
3000036    36490.0
3000037    12990.0
3000038    26998.0
3000039    19900.0
Name: price, Length: 3000040, dtype: float64

In [171]:
used_car_data.seller_rating = used_car_data.seller_rating.fillna(4.)

In [172]:
used_car_data.sp_name

0                                          Flagship Chrysler
1                                        Land Rover San Juan
2                                           FIAT de San Juan
3                                        Land Rover San Juan
4                                        Land Rover San Juan
                                 ...                        
3000035    CarMax Fairfield - Now offering Curbside Picku...
3000036                                       Team Chevrolet
3000037              Hanlees Chrysler Dodge Jeep Ram of Napa
3000038    CarMax Fairfield - Now offering Curbside Picku...
3000039                                      Napa Nissan Inc
Name: sp_name, Length: 3000040, dtype: object

In [181]:
used_car_data.vehicle_damage_category.dropna()

Series([], Name: vehicle_damage_category, dtype: float64)

In [183]:
used_car_data.wheel_system.value_counts()

FWD    1261367
AWD     695731
4WD     584524
RWD     190757
4X2     120929
Name: wheel_system, dtype: int64

In [185]:
used_car_data.width = used_car_data.width.replace('--', float('nan'))
used_car_data.width.str.endswith(' in').all()

True

In [186]:
used_car_data.width = used_car_data.width.str.replace(' in', '').astype('float')

In [187]:
used_car_data.year

0          2019
1          2020
2          2016
3          2020
4          2020
           ... 
3000035    2018
3000036    2020
3000037    2016
3000038    2017
3000039    2017
Name: year, Length: 3000040, dtype: int64

In [192]:
model_age = year_listed - used_car_data.year

In [194]:
used_car_data.wheelbase = used_car_data.wheelbase.replace('--', float('nan'))
used_car_data.wheelbase.str.endswith(' in').all()

True

In [195]:
used_car_data.wheelbase = used_car_data.wheelbase.str.replace(' in', '').astype('float')

In [202]:
used_car_data.body_type.hasnans

True

In [207]:
le_bed = preprocessing.LabelEncoder()
le_bed.fit(used_car_data.bed)
le_bed.classes_

array(['Long', 'Regular', 'Short', nan], dtype=object)

In [209]:
used_car_data.bed = le_bed.transform(used_car_data.bed)

In [210]:
def trf_cat(feat):
    le = preprocessing.LabelEncoder()
    le.fit(used_car_data[feat])
    print(le.classes_)
    print('Good?')
    if input() == 'y':
        used_car_data[feat] = le.transform(used_car_data[feat])
        print('Done')

        

In [211]:
trf_cat('body_type')

['Convertible' 'Coupe' 'Hatchback' 'Minivan' 'Pickup Truck'
 'SUV / Crossover' 'Sedan' 'Van' 'Wagon' nan]
Good?


 y


Done


In [212]:
trf_cat('cabin')

['Crew Cab' 'Extended Cab' 'Large Crew Cab' 'Regular Cab' nan]
Good?


 y


Done


In [213]:
trf_cat('city')

['Albuquerque' 'Alexandria' 'Arlington' 'Atlanta' 'Auburn' 'Aurora'
 'Austin' 'Baton Rouge' 'Birmingham' 'Bloomington' 'Carrollton' 'Chandler'
 'Charleston' 'Charlotte' 'Chicago' 'Cincinnati' 'Clarksville' 'Cleveland'
 'Colorado Springs' 'Columbia' 'Columbus' 'Concord' 'Dallas' 'Denver'
 'Duluth' 'El Paso' 'Englewood' 'Fort Lauderdale' 'Fort Myers'
 'Fort Worth' 'Franklin' 'Fremont' 'Gainesville' 'Gilbert' 'Glendale'
 'Grand Rapids' 'Greenville' 'Henderson' 'Houston' 'Indianapolis'
 'Jackson' 'Jacksonville' 'Kansas City' 'Knoxville' 'Lafayette'
 'Las Vegas' 'Lexington' 'Littleton' 'Los Angeles' 'Louisville' 'Madison'
 'Manchester' 'Marietta' 'Memphis' 'Mesa' 'Miami' 'Milwaukee'
 'Minneapolis' 'Oklahoma City' 'Omaha' 'Ontario' 'Orlando' 'Pensacola'
 'Peoria' 'Phoenix' 'Pittsburgh' 'Plano' 'Raleigh' 'Reno' 'Richmond'
 'Rochester' 'Roseville' 'Sacramento' 'Saint Louis' 'Salt Lake City'
 'San Antonio' 'San Diego' 'San Jose' 'Sanford' 'Scottsdale' 'Spring'
 'Springfield' 'Stafford' 'Tampa' 

 y


Done


In [214]:
trf_cat('engine_cylinders')

['H4' 'H6' 'I2' 'I3' 'I4' 'I5' 'I6' 'R2' 'V1' 'V6' 'V8' 'W1' 'W8' nan]
Good?


 y


Done


In [215]:
trf_cat('franchise_make')

['Acura' 'Alfa Romeo' 'Aston Martin' 'Audi' 'BMW' 'Bentley' 'Buick'
 'Cadillac' 'Chevrolet' 'Chrysler' 'Dodge' 'FIAT' 'Ferrari' 'Ford'
 'Freightliner' 'GMC' 'Genesis' 'Honda' 'Hyundai' 'INFINITI' 'Jaguar'
 'Jeep' 'Kia' 'Lamborghini' 'Land Rover' 'Lexus' 'Lincoln' 'Lotus' 'MINI'
 'Maserati' 'Mazda' 'McLaren' 'Mercedes-Benz' 'Mitsubishi' 'Nissan'
 'Pagani' 'Porsche' 'RAM' 'Rolls-Royce' 'Rover' 'SRT' 'Scion' 'Shelby'
 'Subaru' 'Toyota' 'Volkswagen' 'Volvo' 'smart' nan]
Good?


 y


Done


In [217]:
used_car_data.fuel_type = used_car_data.fuel_type.fillna('Gasoline')

In [218]:
trf_cat('fuel_type')

['Biodiesel' 'Compressed Natural Gas' 'Diesel' 'Electric'
 'Flex Fuel Vehicle' 'Gasoline' 'Hybrid' 'Propane']
Good?


 y


Done


In [219]:
trf_cat('listing_color')

['BLACK' 'BLUE' 'BROWN' 'GOLD' 'GRAY' 'GREEN' 'ORANGE' 'PINK' 'PURPLE'
 'RED' 'SILVER' 'TEAL' 'UNKNOWN' 'WHITE' 'YELLOW']
Good?


 y


Done


In [223]:
used_car_data.transmission = used_car_data.transmission.fillna('A')

In [224]:
trf_cat('transmission')

['A' 'CVT' 'Dual Clutch' 'M']
Good?


 y


Done


In [227]:
used_car_data.wheel_system = used_car_data.wheel_system.fillna('FWD')

In [228]:
trf_cat('wheel_system')

['4WD' '4X2' 'AWD' 'FWD' 'RWD']
Good?


 y


Done


In [234]:
used_car_data.engine_cylinders.value_counts()

4     1507448
9      818732
10     386953
13     100581
0       65955
3       54304
6       51776
5        5446
1        4686
8        2611
2         897
11        583
7          65
12          3
Name: engine_cylinders, dtype: int64

In [235]:
used_car_data.engine_cylinders = used_car_data.engine_cylinders.replace(13, 4)

In [236]:
used_car_data.cabin.value_counts()

4    2936507
0      51083
1       7960
3       2966
2       1524
Name: cabin, dtype: int64

In [249]:
sum(used_car_data.mileage.isna())

144387

In [252]:
used_car_data.mileage.median()

8267.0

In [253]:
used_car_data.mileage = used_car_data.mileage.fillna(10000)

In [254]:
processed_data = pd.DataFrame(used_car_data[
    ['price',
     'bed', 'body_type', 'cabin', 'city', 'engine_cylinders', 'franchise_make', 'fuel_type', 'listing_color', 'transmission', 'wheel_system',
     'fleet', 'frame_damaged', 'franchise_dealer', 'has_accidents', 'isCab', 'is_cpo', 'is_new', 'salvage',
     'back_legroom', 'bed_length', 'city_fuel_economy', 'daysonmarket', 'engine_displacement', 'front_legroom', 'fuel_tank_volume', 'height', 'highway_fuel_economy', 'horsepower', 'length', 'maximum_seating', 'mileage', 'owner_count', 'seller_rating', 'width', 'wheelbase']
])
processed_data['year_listed'] = year_listed
processed_data['month_listed'] = month_listed
processed_data['model_age'] = model_age
processed_data

,price,bed,body_type,cabin,city,engine_cylinders,franchise_make,fuel_type,listing_color,transmission,...,length,maximum_seating,mileage,owner_count,seller_rating,width,wheelbase,year_listed,month_listed,model_age
0,23141.0,3,5,4,55,4,21,5,14,0,...,166.6,5.0,7.0,1.0,2.800000,79.6,101.2,2019,4,0
1,46500.0,3,5,4,55,4,24,5,0,0,...,181.0,7.0,8.0,1.0,3.000000,85.6,107.9,2020,2,0
2,46995.0,3,6,4,55,0,11,5,12,3,...,180.9,5.0,10000.0,3.0,4.000000,78.9,104.3,2017,4,1
3,67430.0,3,5,4,55,9,24,5,4,0,...,195.1,7.0,11.0,1.0,3.000000,87.4,115.0,2020,2,0
4,48880.0,3,5,4,55,4,24,5,0,0,...,181.0,7.0,7.0,1.0,3.000000,85.6,107.9,2020,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000035,17998.0,3,5,4,31,4,48,5,10,0,...,183.1,5.0,41897.0,1.0,4.272727,72.6,107.3,2020,8,2
3000036,36490.0,3,5,4,31,9,8,5,0,0,...,204.3,8.0,5.0,1.0,4.533333,78.6,120.9,2020,3,0
3000037,12990.0,3,6,4,31,4,21,5,4,0,...,191.7,5.0,57992.0,2.0,4.142857,83.5,112.2,2020,6,4
3000038,26998.0,3,6,4,31,4,48,2,5,0,...,183.9,5.0,27857.0,1.0,4.272727,81.7,111.6,2020,9,3


In [255]:
processed_data.to_parquet("processed_ucd.parquet", compression=None)

In [256]:
processed_data.to_csv("processed_ucd.csv")